In [2]:
import numpy as np
import mne
import gym

In [3]:
import pywt
import scipy.signal
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1).reshape(-1, 1)

def stddev(x):
    return np.std(x, axis=-1).reshape(-1, 1)

def peaktopeak(x):
    return np.ptp(x, axis=-1).reshape(-1, 1)

def variance(x):
    return np.var(x, axis=-1).reshape(-1, 1)

def mini(x):
    return np.min(x, axis=-1).reshape(-1, 1)

def maxi(x):
    return np.max(x, axis=-1).reshape(-1, 1)

def argmini(x):
    return np.argmin(x, axis=-1).reshape(-1, 1)

def argmaxi(x):
    return np.argmax(x, axis=-1).reshape(-1, 1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1)).reshape(-1, 1)

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1).reshape(-1, 1)

def skewness(x):
    return stats.skew(x, axis=-1).reshape(-1, 1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1).reshape(-1, 1)

def concat_features(x):
    features = np.concatenate(
        (
            peaktopeak(x),
            rms(x),
            abs_diff_signal(x),
            skewness(x),
            kurtosis(x),
            variance(x),
            mean(x),
            stddev(x)
        ),
        axis=1
    )
    return features

def apply_cwt(data, scales, wavelet_name='morl'):
    """
    Apply Continuous Wavelet Transform (CWT) to EEG data.

    :param data: EEG data in CSP space with shape (components, timepoints)
    :param scales: Scales for CWT
    :param wavelet_name: Name of the mother wavelet for CWT
    :return: CWT coefficients
    """
    cwt_coeffs = np.array([pywt.cwt(data[i, :], scales, wavelet_name)[0] for i in range(data.shape[0])])
    return cwt_coeffs

    
def featuresarray_load(data_array):
    features = []
    fs = 250
    for d in data_array:
        
       
        alpha = mne.filter.filter_data(d, sfreq=fs, l_freq=8, h_freq=12,verbose=False)
        beta = mne.filter.filter_data(d, sfreq=fs, l_freq=12, h_freq=30,verbose=False)
        
        alph_ftrs = concat_features(alpha)
        beta_ftrs = concat_features(beta)
        
        #nperseg = 256
        
        
        _,p=scipy.signal.welch(beta, fs=fs,average='median',nfft = 512)
        _,p2=scipy.signal.welch(alpha, fs=fs,average='median',nfft = 512)
        

        res = np.mean([alph_ftrs,beta_ftrs],axis=0)
        #print('p',p.shape,res.shape)
        res = np.concatenate((res,p,p2),axis=1)
        #print(res.shape)
        features.append(res)
    return np.array(features)

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from mne.decoding import CSP
import mne
from mne.decoding import CSP
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline

event_ids = [1,2,3,4, 5]  
event_id_to_label = {}
for i in range(len(event_ids)):
    event_id_to_label[i] = event_ids[i]
print(event_id_to_label)
path = 'c:/dev/eeg/eegfrontendpage/eeg_tool/src/data/formatted/S03.fif'
raw = mne.io.read_raw_fif(path, preload=True)
print('eventids',mne.events_from_annotations(raw)[1])
ica = mne.preprocessing.ICA(n_components=len(raw.info['ch_names']), random_state=42, max_iter=1000)
ica.fit(raw)
ica.apply(raw)
csp_filters = {} 
events = mne.events_from_annotations(raw)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}
Opening raw data file c:/dev/eeg/eegfrontendpage/eeg_tool/src/data/formatted/S03.fif...
Isotrak not found
    Range : 0 ... 11248 =      0.000 ...    44.992 secs
Ready.


C:\Users\ncvn\AppData\Local\Temp\ipykernel_18448\300687801.py:23: RuntimeWarning: This filename (c:/dev/eeg/eegfrontendpage/eeg_tool/src/data/formatted/S03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(path, preload=True)


Reading 0 ... 11248  =      0.000 ...    44.992 secs...
Used Annotations descriptions: ['blinking', 'jaw', 'left', 'relax', 'right']
eventids {'blinking': 1, 'jaw': 2, 'left': 3, 'relax': 4, 'right': 5}
Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by number: 3 components
Fitting ICA took 0.0s.
Applying ICA to Raw instance
    Transforming to ICA space (3 components)
    Zeroing out 0 ICA components
    Projecting back using 3 PCA components
Used Annotations descriptions: ['blinking', 'jaw', 'left', 'relax', 'right']


C:\Users\ncvn\AppData\Local\Temp\ipykernel_18448\300687801.py:26: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


In [5]:
raw

<Raw | S03.fif, 3 x 11249 (45.0 s), ~272 kB, data loaded>

In [7]:
# mapping = {
#     'EEG 1': 'FCz',
#     'EEG 2': 'C3',
#     'EEG 3': 'FC1',
#     'EEG 4': 'CPz',
#     'EEG 5': 'C2',
#     'EEG 6': 'C4'
# }

# raw.rename_channels(mapping)
# Load the standard 10-20 montage
montage = mne.channels.make_standard_montage('standard_1020')

# Apply the montage to your raw data
raw.set_montage(montage)

<Raw | S03.fif, 3 x 11249 (45.0 s), ~274 kB, data loaded>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft
import tensorflow as tf

all_epochs = mne.Epochs(raw, events[0], event_id=[1,2,3,4, 5],  # No specific event_id filtering
                        tmin=-0.5, tmax=5.5, baseline=(-0.5,1), preload=True)
all_epochs.pick_types(meg=False, eeg=True)

X_all = all_epochs.get_data()
save_copy = X_all.copy()

event_ids_all = all_epochs.events[:, -1]
print(X_all.shape,event_ids_all.shape)
csp_filter_objects = {}
ncomp = 4
csp_transformed_data = {}
for event_id in [1,2,3,4,5]:
    y = (event_ids_all == event_id).astype(int)
    if np.unique(y).size < 2:
        print(f"Skipping event_id {event_id}.")
        continue

    csp = CSP(n_components=ncomp, norm_trace=False, transform_into='csp_space')
    csp.fit(X_all, y)
    csp.plot_patterns(info=raw.info,
                  components=[0,1,2,3], ch_type='eeg')
    csp_transformed_data[event_id] = csp.transform(X_all)
    csp_filter_objects[event_id] = csp

import joblib
joblib.dump(csp_filter_objects, 'csp_filters_ovr.pkl')
print("CSP filters saved successfully.")

print('CSP FILTERS DICT:',csp_transformed_data)
# Combine CSP features for each trial based on its label
n_trials = len(X_all)  # Number of trials
n_components = ncomp       # Number of CSP components (assuming 3 for this example)
n_time_points = csp_transformed_data[1].shape[2]   # Number of time points in the transformed CSP data

# Initialize the combined_features array to hold CSP features for all trials
combined_features = np.zeros((n_trials, n_components, n_time_points))

# Loop through each trial and assign the CSP-transformed data
for i, label in enumerate(event_ids_all):
    # Fetch the CSP features for the current trial and class label
    # Adjust the indexing based on how your labels and csp_transformed_data are structured
    csp_features_for_label = csp_transformed_data.get(label, None)

    # Check if the label exists in the dictionary and if the index is within bounds
    if csp_features_for_label is not None and i < len(csp_features_for_label):
        combined_features[i, :, :] = csp_features_for_label[i]

print(combined_features.shape)

print(combined_features.shape)
y = np.zeros((X_all.shape[0], len(event_ids)))  

for i, event_id in enumerate([1,2,3,4,5]):
    binary_labels = (event_ids_all == event_id).astype(int)
    y[:, i] = binary_labels  
print(y)

y_flattened = np.argmax(y, axis=1)
print(y_flattened)
print(y_flattened.tolist().count(0),y_flattened.tolist().count(1),y_flattened.tolist().count(2),y_flattened.tolist().count(3))
# clf = Pipeline([('scaler',StandardScaler()),('SVC', SVC())])
print('features shape: ',combined_features.shape,y_flattened.shape)
# scores = cross_val_score(clf, combined_features, y_flattened, cv=10, scoring='accuracy')
# print("Multiclass classification accuracy: %f" % scores.mean())

ftrs = featuresarray_load(combined_features)

print('features shape: ',ftrs.shape,y_flattened.shape)
X_train, X_test, y_train, y_test = train_test_split(ftrs, y_flattened, train_size=0.9, random_state=42, stratify=y_flattened)


from keras.layers import PReLU, Conv1D, Dropout, SpatialDropout1D, MaxPooling1D, GlobalMaxPooling1D, Layer, AveragePooling1D, LSTM, Reshape, BatchNormalization
from keras.regularizers import l1_l2

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
GLOBAL_SHAPE_LENGTH = ftrs.shape[2]
# model = tf.keras.models.load_model('1dcnnlstm_model_88p_acc.keras')
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=32)


Not setting metadata
9 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 9 events and 1501 original time points ...
2 bad epochs dropped
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
(7, 3, 1501) (7,)
Computing rank from data with rank=None
    Using tolerance 25 (2.2e-16 eps * 3 dim * 3.8e+16  max singular value)
    Estimated rank (data): 3
    data: rank 3 computed from 3 data channels with 0 projectors
Reducing data rank from 3 -> 3
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


ValueError: Times should be between 0.0 and 2.0.

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [9]:
# model.evaluate(X_test,y_test)


In [ ]:
joblib.dump(scaler, 'scaler.pkl')
print("Scaler saved successfully.")

In [ ]:
csp_transformed_data[4].shape

In [12]:
# import numpy as np
# import time
# import socket
# from stable_baselines3 import DQN
# from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
# from mne.decoding import CSP
# import joblib
# import tensorflow as tf
# # Import your feature extraction function
# # Ensure that featuresarray_load is correctly defined or imported above
# # from your_feature_module import featuresarray_load  # Adjust the import as necessary

# # Load the pre-trained DQN model
# model = tf.keras.models.load_model('1dcnnlstm_model_88p_acc.keras')
# #DQN.load("dqn_plasticity_final")  # Ensure the path is correct

# # Load the pre-trained CSP filters
# csp_filters = joblib.load('csp_filters_ovr.pkl')  # Ensure this path is correct
# print("CSP filters loaded for real-time use.",csp_filters)

# # Load the scaler used during training
# scaler = joblib.load('scaler.pkl')  # Ensure this path is correct
# print("Scaler loaded for real-time use.")

# # Circular Buffer for handling 6 EEG channels
# class MultiChannelCircularBuffer:
#     def __init__(self, num_channels, buffer_size):
#         self.buffer = np.zeros((num_channels, buffer_size))
#         self.buffer_size = buffer_size
#         self.index = 0

#     def add_data(self, data):
#         num_samples = data.shape[1]
#         if self.index + num_samples <= self.buffer_size:
#             self.buffer[:, self.index:self.index + num_samples] = data
#         else:
#             end_index = (self.index + num_samples) % self.buffer_size
#             self.buffer[:, self.index:] = data[:, :self.buffer_size - self.index]
#             self.buffer[:, :end_index] = data[:, self.buffer_size - self.index:]
#         self.index = (self.index + num_samples) % self.buffer_size

#     def get_window(self, window_size):
#         if self.index < window_size:
#             return np.concatenate((self.buffer[:, -window_size + self.index:], self.buffer[:, :self.index]), axis=1)
#         else:
#             return self.buffer[:, self.index - window_size:self.index]

# # Initialize buffer and window parameters
# num_channels = 6
# buffer_size = 1250 * 10
# window_size = 1250
# num_samples = 1250  # 5 seconds of data at 250 Hz

# # Initialize the multi-channel buffer
# eeg_buffer = MultiChannelCircularBuffer(num_channels, buffer_size)

# # Set up BrainFlow for OpenBCI (Cyton board as an example)
# params = BrainFlowInputParams()
# params.serial_port = 'COM9'  # Replace with your actual COM port for OpenBCI
# board = BoardShim(BoardIds.CYTON_BOARD.value, params)
# board.prepare_session()
# board.start_stream()

# def map_action_to_command(action):
#     action_mapping = {
#         0: 'L',
#         1: 'R',
#         2: 'F',
#         3: 'B'
#     }
#     return action_mapping.get(action, 'STOP')

# # Set up socket connection to Arduino

# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# def send_command_to_arduino(command):
#     try:
#         arduino_ip = '192.168.1.168'  # Replace with your Arduino's IP
#         arduino_port = 80         # Replace with your Arduino's listening port
        
#         try:
#             sock.connect((arduino_ip, arduino_port))
#             print("Connected to Arduino")
#         except Exception as e:
#             print(f"Failed to connect to Arduino: {e}")
        
#         # HTTP GET request
#         request = f"GET /{command} HTTP/1.1\r\nHost: {arduino_ip}\r\n\r\n"
#         sock.send(request.encode())
        
#         # Close the connection
#         sock.close()
#         print(f"Sent command: {command}")
        
#     except Exception as e:
#         print(f"Error sending command: {e}")

# # Real-time EEG data acquisition and prediction loop
# try:
#     while True:
#         data_count = 0
#         data_count = board.get_board_data_count()
#         print('data count',data_count)
#         if data_count >= num_samples:
#             # Retrieve the latest num_samples data points
#             data = board.get_current_board_data(num_samples)
#             print(f"Data shape: {data.shape}")

#             # Extract EEG data from the first 6 channels
#             eeg_channels = BoardShim.get_eeg_channels(BoardIds.CYTON_BOARD.value)
#             eeg_data = data[eeg_channels[:6], :]
#             eeg_buffer.add_data(eeg_data)  # Add to buffer

#             # Check if buffer has enough data for a prediction window
#             if eeg_buffer.index >= window_size:
#                 window_data = eeg_buffer.get_window(window_size).reshape(1, num_channels, window_size)

#                 # Initialize an empty list to collect CSP features for all classes
#                 csp_features_list = []

#                 # Apply CSP transform for each class (One-vs-Rest)
#                 for class_id, csp in csp_filter_objects.items():
#                     # CSP expects data in shape (n_trials, n_channels, n_times)
#                     transformed = csp.transform(window_data)  # Shape: (n_trials, n_components)
#                     csp_features_list.append(transformed)

#                 # Concatenate CSP features from all classes
#                 # Resulting shape: (n_trials, n_classes * n_components)
#                 csp_ftrs = np.concatenate(csp_features_list, axis=1)
                
#                 print('cspftrs shape:',csp_ftrs.shape)

#                 # Reshape csp_ftrs to match featuresarray_load expected input
#                 # Assuming featuresarray_load expects (n_samples, n_features)
#                 #csp_ftrs_reshaped = csp_ftrs.reshape(1, -1)  # Shape: (1, n_features)

#                 # Extract features using featuresarray_load
#                 ftrs = featuresarray_load(csp_ftrs)  # Adjust if necessary

#                 # Apply the pre-fitted scaler
#                 ftrs = scaler.transform(ftrs.reshape(-1, ftrs.shape[-1])).reshape(ftrs.shape)

#                 print(ftrs.shape)
#                 # Predict action with the DQN model
#                 action, _states = model.predict(ftrs[:, 0, :], deterministic=True)
#                 print(action)
#                 # Map action to command and send to Arduino
#                 command = map_action_to_command(int(action))
#                 send_command_to_arduino(command)

#             time.sleep(0.1)  # Control loop frequency (e.g., 10 Hz)
#         else:
#             pass

# except KeyboardInterrupt:
#     print("Stopped by user.")
# finally:
#     sock.close()
#     board.stop_stream()
#     board.release_session()


In [13]:
from stable_baselines3.dqn.policies import DQNPolicy
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback, CheckpointCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy

class KerasDQNPolicy(DQNPolicy):
    def __init__(self, *args, **kwargs):
        super(KerasDQNPolicy, self).__init__(*args, **kwargs)
        self.keras_model = Sequential([
            Reshape((GLOBAL_SHAPE_LENGTH,ncomp)),
            BatchNormalization(),

            Conv1D(32, kernel_size=3),
            BatchNormalization(),
            PReLU(),
            

            MaxPooling1D(pool_size=2),
            SpatialDropout1D(0.1),

            Conv1D(64, kernel_size=3),
            BatchNormalization(),
            PReLU(),
            AveragePooling1D(pool_size=2),
            SpatialDropout1D(0.1),

            LSTM(64, activation='tanh', recurrent_regularizer=l1_l2(l1=0.01, l2=0.01),return_sequences=True),
            BatchNormalization(),
            GlobalMaxPooling1D(),
            BatchNormalization(),
            Dense(units=64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
            BatchNormalization(),
            Dropout(0.1),
            Dense(units=32, activation='relu'),
            BatchNormalization(),
            Dropout(0.1),
            Dense(units=5, activation='linear')
        ])
    def q_values(self, obs):
        return self.keras_model.predict(np.array(obs))

In [14]:
import random
class Plasticity(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(X_train, y_train), random=True):
        super(Plasticity, self).__init__()
        self.action_space = gym.spaces.Discrete(5)  # 4 actions
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf,
                                                shape=(ncomp, GLOBAL_SHAPE_LENGTH),
                                                dtype=np.float32)
        self.images_per_episode = images_per_episode
        self.step_count = 0
        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = self.calculate_reward(action)
        obs = self._next_obs()
        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True
        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0
        return self._next_obs()

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]
        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])
            self.dataset_idx = (self.dataset_idx) % (len(X_train))
            # if self.dataset_idx >= len(self.x):
            #     raise StopIteration()
        return obs

    def calculate_reward(self, action):
        if action == self.expected_action:
            return 1.0
        else:
            return -1.0  # Negative reward for incorrect actions


In [ ]:
# Instantiate the environment
env = Plasticity(images_per_episode=1, dataset=(X_train, y_train), random=True)

# Initialize the DQN agent with custom policy
model = DQN(
    KerasDQNPolicy,  # Use custom policy instead of "MlpPolicy"
    env,
    verbose=1,
    learning_rate=5e-4,
    buffer_size=50000,
    learning_starts=100,
    batch_size=32,
    gamma=0.99,
    train_freq=4,
    target_update_interval=200,
    exploration_fraction=0.1,
    exploration_final_eps=0.02,
    tensorboard_log="./dqn_plasticity_tensorboard/"
)

# Define callbacks
checkpoint_callback = CheckpointCallback(
    save_freq=1000,
    save_path='./models/',
    name_prefix='dqn_plasticity'
)

eval_env = Plasticity(images_per_episode=1, dataset=(X_test, y_test), random=False)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path='./logs/',
    log_path='./logs/',
    eval_freq=1000,
    deterministic=True,
    render=False
)

class CustomCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(CustomCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        if self.n_calls % 1000 == 0:
            print(f"Step: {self.n_calls}")
        return True

custom_callback = CustomCallback()

In [ ]:
# Train the agent
model.learn(
    total_timesteps=2500,
    callback=[checkpoint_callback, eval_callback, custom_callback]
)

# Save the trained model
model.save("dqn_plasticity_final")


In [ ]:
eval_env.x.shape

In [ ]:

# Load the model (if needed)
model = DQN.load("dqn_plasticity_final", env=env)
eval_env = Plasticity(images_per_episode=1, dataset=(X_test, y_test), random=False)
# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100,deterministic=True)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

In [19]:
# import numpy as np
# import time
# import joblib
# import tensorflow as tf
# from mne.decoding import CSP
# from sklearn.preprocessing import StandardScaler
# from keras.models import Sequential
# from keras.layers import (
#     PReLU, Conv1D, Dropout, SpatialDropout1D, MaxPooling1D, 
#     GlobalMaxPooling1D, BatchNormalization, LSTM, Dense, Reshape
# )
# from keras.regularizers import l1_l2

# # Import your feature extraction function
# # Ensure that featuresarray_load is correctly defined or imported above
# # from your_feature_module import featuresarray_load  # Adjust the import as necessary


# # ==========================
# # Load Pre-trained Models and Scalers
# # ==========================

# # Load the pre-trained Keras model
# cnnlsmt_model = tf.keras.models.load_model('1dcnnlstm_model_88p_acc.keras')

# # Load the pre-trained CSP filters (assuming One-vs-Rest CSP)
# csp_filters = joblib.load('csp_filters_ovr.pkl')  # Ensure this path is correct
# print("CSP filters loaded for simulated use.",csp_filter_objects)

# # Load the scaler used during training
# scaler = joblib.load('scaler.pkl')  # Ensure this path is correct
# print("Scaler loaded for simulated use.")

# # ==========================
# # Define the Circular Buffer
# # ==========================

# class MultiChannelCircularBuffer:
#     def __init__(self, num_channels, buffer_size):
#         self.buffer = np.zeros((num_channels, buffer_size))
#         self.buffer_size = buffer_size
#         self.index = 0

#     def add_data(self, data):
#         num_samples = data.shape[1]
#         if self.index + num_samples <= self.buffer_size:
#             self.buffer[:, self.index:self.index + num_samples] = data
#         else:
#             end_index = (self.index + num_samples) % self.buffer_size
#             self.buffer[:, self.index:] = data[:, :self.buffer_size - self.index]
#             self.buffer[:, :end_index] = data[:, self.buffer_size - self.index:]
#         self.index = (self.index + num_samples) % self.buffer_size

#     def get_window(self, window_size):
#         if self.index < window_size:
#             return np.concatenate((self.buffer[:, -window_size + self.index:], self.buffer[:, :self.index]), axis=1)
#         else:
#             return self.buffer[:, self.index - window_size:self.index]

# # ==========================
# # Simulate EEG Data Generation
# # ==========================

# def generate_simulated_eeg(num_channels, num_samples, fs=250):
#     """
#     Generate simulated EEG data with alpha and beta rhythms.
    
#     :param num_channels: Number of EEG channels
#     :param num_samples: Number of samples per channel
#     :param fs: Sampling frequency in Hz
#     :return: Simulated EEG data as a numpy array of shape (num_channels, num_samples)
#     """
#     t = np.linspace(0, num_samples / fs, num_samples, endpoint=False)
#     eeg_data = np.zeros((num_channels, num_samples))
    
#     # Define frequency components for each channel
#     for ch in range(num_channels):
#         # Simulate alpha (10 Hz) and beta (20 Hz) rhythms with some random noise
#         alpha_freq = 10 + np.random.uniform(-1, 1)
#         beta_freq = 20 + np.random.uniform(-1, 1)
#         eeg_data[ch, :] = (
#             np.sin(2 * np.pi * alpha_freq * t) +
#             0.5 * np.sin(2 * np.pi * beta_freq * t) +
#             0.3 * np.random.randn(num_samples)  # Additive Gaussian noise
#         )
#     return eeg_data

# # ==========================
# # Define Action Mapping (No Socket)
# # ==========================

# def map_action_to_command(action):
#     """
#     Map the model's predicted action to a command.
    
#     :param action: Integer representing the action
#     :return: Command string
#     """
#     action_mapping = {
#         0: 'L',    # Left
#         1: 'R',    # Right
#         2: 'F',    # Forward
#         3: 'B'     # Backward
#     }
#     return action_mapping.get(action, 'STOP')

# # ==========================
# # Initialize Buffer and Parameters
# # ==========================

# num_channels = 6
# fs = 250  # Sampling frequency
# buffer_size = fs * 10  # 10 seconds buffer
# window_size = fs * 5   # 5 seconds window
# num_samples = fs * 5    # 5 seconds of data per iteration

# # Initialize the multi-channel buffer
# eeg_buffer = MultiChannelCircularBuffer(num_channels, buffer_size)

# # ==========================
# # Initialize CSP Filters (One-vs-Rest)
# # ==========================

# # Assuming 'csp_filters' is a dictionary with class IDs as keys and CSP objects as values
# csp_filter_objects = csp_filters  # Rename for clarity

# # ==========================
# # Simulated Prediction Loop
# # ==========================

# try:
#     while True:
#         # Simulate data acquisition delay
#         time.sleep(0.1)  # Control loop frequency (e.g., 10 Hz)
        
#         # Generate simulated EEG data
#         simulated_data = generate_simulated_eeg(num_channels, num_samples, fs=fs)
#         print(f"Simulated Data Shape: {simulated_data.shape}")
        
#         # Add simulated data to buffer
#         eeg_buffer.add_data(simulated_data)
        
#         # Check if buffer has enough data for a prediction window
#         if eeg_buffer.index >= window_size:
#             # Retrieve the latest window_size data points
#             window_data = eeg_buffer.get_window(window_size).reshape(1, num_channels, window_size)
#             print(f"Window Data Shape: {window_data.shape}")
            
#             # Initialize an empty list to collect CSP features for all classes
#             csp_features_list = []
            
#             # Apply CSP transform for each class (One-vs-Rest)
#             for class_id, csp in csp_filter_objects.items():
#                 # CSP expects data in shape (n_trials, n_channels, n_times)
#                 transformed = csp.transform(window_data)  # Shape: (n_trials, n_components)
#                 csp.plot_patterns(info=raw.info,
#                   components=list(range(ncomp)), ch_type='eeg')
#                 csp_features_list.append(transformed)
            
#             # Concatenate CSP features from all classes
#             # Resulting shape: (n_trials, n_classes * n_components)
#             csp_ftrs = np.concatenate(csp_features_list, axis=1)
#             print(f"CSP Features Shape: {csp_ftrs.shape}")
            
#             # Extract features using featuresarray_load
#             ftrs = featuresarray_load(csp_ftrs)
#             print(f"Extracted Features Shape: {ftrs.shape}")
#             predictions = []
#             # Apply the pre-fitted scaler
#             print('Entering ensenble loop...')
#             for i in range(0,16,4):
#                 ftrs_scaled = (scaler.transform(ftrs.reshape(-1, ftrs.shape[-1])).reshape(ftrs.shape))
#                 ftrs_scaled = ftrs_scaled[:,i:i+4,:]
                
#                 print(ftrs_scaled.shape)
#                 # Predict action with the pre-trained model
#                 # Assuming the model expects input shape compatible with ftrs_scaled
#                 # Here, adjust the input shape as per your model's requirement
#                 # For example, if model expects (batch_size, time_steps, features), reshape accordingly
#                 # Here, assuming ftrs_scaled is already in the correct shape
#                 a,b = model.predict(ftrs_scaled)
#                 print('a',a,'b',b)
#                 print('prediction',a[0])
#                 predictions.append(a[0])
                
#                 # Map action to command
#             predictions = np.array(predictions)
#             print('prediction array', predictions)
#             action = np.bincount(predictions).argmax()
#             print(f"Predicted Action: {action}")
#             command = map_action_to_command(int(action))
#             print(f"Mapped Command: {command}")
            
#             # Here, instead of sending the command to Arduino, we simply print it
#             # You can add additional handling if needed
#             # send_command_to_arduino(command)  # Removed socket communication

# except KeyboardInterrupt:
#     print("Stopped by user.")


In [26]:
# import numpy as np
# import time
# import socket
# from stable_baselines3 import DQN
# from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
# from mne.decoding import CSP
# import joblib
# import tensorflow as tf

# # Constants
# OVERLAP_RATIO = 0.8
# SMOOTHING_WINDOW = 5
# MIN_CONFIDENCE = 0.3
# MAX_CONFIDENCE = 0.8
# COMMAND_DURATION = 0.5

# # Load the pre-trained DQN model
# model = DQN.load("dqn_plasticity_final")
# print("DQN model loaded.")

# # Load the pre-trained CSP filters
# csp_filters = joblib.load('csp_filters_ovr.pkl')
# print("CSP filters loaded for real-time use.")

# # Load the scaler used during training
# scaler = joblib.load('scaler.pkl')
# print("Scaler loaded for real-time use.")

# class MultiChannelCircularBuffer:
#     def __init__(self, num_channels, buffer_size, window_size=1250, overlap_ratio=0.8):
#         self.buffer = np.zeros((num_channels, buffer_size))
#         self.buffer_size = buffer_size
#         self.window_size = window_size  # Size of analysis window (e.g., 5s = 1250 samples)
#         self.overlap_samples = int(window_size * overlap_ratio)  # Number of samples to overlap
#         self.step_size = window_size - self.overlap_samples  # Number of new samples needed
#         self.index = 0
#         self.last_window_end = 0
        
#         print(f"Initialized circular buffer:")
#         print(f"- Buffer size: {buffer_size} samples")
#         print(f"- Window size: {window_size} samples ({window_size/250:.1f}s)")
#         print(f"- Overlap: {overlap_ratio*100:.0f}% ({self.overlap_samples} samples)")
#         print(f"- Step size: {self.step_size} samples ({self.step_size/250:.3f}s)")

#     def add_data(self, data):
#         """Add new data to buffer and return True if enough new samples for next window"""
#         num_samples = data.shape[1]
        
#         # Add data to buffer
#         if self.index + num_samples <= self.buffer_size:
#             self.buffer[:, self.index:self.index + num_samples] = data
#         else:
#             # Handle wrap-around
#             end_index = (self.index + num_samples) % self.buffer_size
#             self.buffer[:, self.index:] = data[:, :self.buffer_size - self.index]
#             self.buffer[:, :end_index] = data[:, self.buffer_size - self.index:]
            
#         self.index = (self.index + num_samples) % self.buffer_size
        
#         # Check if we have enough new samples since last window
#         samples_since_last = (self.index - self.last_window_end) % self.buffer_size
#         return samples_since_last >= self.step_size

#     def get_window(self):
#         """Get the latest window of data with overlap"""
#         # Update last window position
#         self.last_window_end = self.index
        
#         # Calculate start position for window
#         start_idx = (self.index - self.window_size) % self.buffer_size
        
#         # Handle wrap-around case
#         if start_idx < self.index:
#             return self.buffer[:, start_idx:self.index]
#         else:
#             return np.concatenate((
#                 self.buffer[:, start_idx:],
#                 self.buffer[:, :self.index]
#             ), axis=1)

# class ArduinoConnection:
#     def __init__(self, host, port):
#         self.host = host
#         self.port = port
#         self.sock = None
        
#     def connect(self):
#         if not self.sock:
#             self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#             self.sock.connect((self.host, self.port))
    
#     def send_command(self, command):
#         if not self.sock:
#             self.connect()
#         try:
#             self.sock.send(f"GET /{command} HTTP/1.1\r\nHost: {self.host}\r\n\r\n".encode())
#             return self.sock.recv(1024).decode()
#         except Exception as e:
#             print(f"Connection error: {e}")
#             self.sock = None
#             return None
    
#     def close(self):
#         if self.sock:
#             self.sock.close()
#             self.sock = None

# def map_action_to_command(action):
#     action_mapping = {
#         0: 'L',
#         1: 'R',
#         2: 'F',
#         3: 'B'
#     }
#     return action_mapping.get(action, 'S')

# # Initialize components
# num_channels = 6
# buffer_size = 1250 * 10  # 10 seconds buffer
# window_size = 1250       # 5 seconds window
# samples_per_read = 250   # Read 1 second of data at a time

# # Initialize the buffer with overlap parameters
# eeg_buffer = MultiChannelCircularBuffer(
#     num_channels=num_channels,
#     buffer_size=buffer_size,
#     window_size=window_size,
#     overlap_ratio=OVERLAP_RATIO
# )

# # Set up BrainFlow
# params = BrainFlowInputParams()
# params.serial_port = 'COM9'  # Replace with your actual COM port
# board = BoardShim(BoardIds.CYTON_BOARD.value, params)
# board.prepare_session()
# print("BrainFlow session prepared.")
# board.start_stream()
# print("Started data stream from board.")

# try:
#     # Initialize Arduino connection
#     arduino = ArduinoConnection('192.168.1.168', 80)
    
#     # Initialize state variables
#     prediction_history = []
#     last_state = None
#     last_command = 'S'
#     last_command_time = time.time()
#     start_time = time.time()
    
#     while True:
#         # Get new data (1 second chunks)
#         data = board.get_current_board_data(samples_per_read)
#         eeg_channels = BoardShim.get_eeg_channels(BoardIds.CYTON_BOARD.value)
#         eeg_data = data[eeg_channels[:6], :]

#         # Add data and check if we have enough for next window
#         if eeg_buffer.add_data(eeg_data):
#             # Get overlapped window and process
#             window_data = eeg_buffer.get_window().reshape(1, num_channels, -1)
#             print(f"Processing window of shape: {window_data.shape}")
            
#             # Initialize CSP features list
#             csp_features_list = []
            
#             # Apply CSP transform for each class
#             for class_id, csp in csp_filters.items():
#                 transformed = csp.transform(window_data)
#                 # if hasattr(raw, 'info'):  # Only plot if raw.info is available
#                 #     csp.plot_patterns(info=raw.info, components=list(range(4)), ch_type='eeg')
#                 csp_features_list.append(transformed)
            
#             # Concatenate CSP features
#             csp_ftrs = np.concatenate(csp_features_list, axis=1)
#             print(f"CSP Features Shape: {csp_ftrs.shape}")
            
#             # Extract features
#             ftrs = featuresarray_load(csp_ftrs)
#             print(f"Extracted Features Shape: {ftrs.shape}")
            
#             predictions = []
#             # Apply scaling and make predictions
#             print('Entering ensemble loop...')
#             for i in range(0, 20, 4):
#                 ftrs_scaled = scaler.transform(ftrs.reshape(-1, ftrs.shape[-1])).reshape(ftrs.shape)
#                 ftrs_scaled = ftrs_scaled[:, i:i+4, :]
#                 print(f"Scaled Features Shape: {ftrs_scaled.shape}")
                
#                 # Get prediction
#                 action, _ = model.predict(ftrs_scaled, deterministic=True)
#                 print(f'Prediction: {action[0]}')
#                 predictions.append(action[0])
            
#             # Aggregate predictions
#             predictions = np.array(predictions)
#             print('Prediction array:', predictions)
#             action = np.bincount(predictions).argmax()
            
#             # Replace the current confidence calculation with:
#             # Replace the current confidence calculation with:
#             q_values = model.q_net(model.q_net.obs_to_tensor(ftrs_scaled)[0])[0].detach().numpy()
#             # Normalize Q-values to be between 0 and 1
#             q_values = (q_values - np.min(q_values)) / (np.max(q_values) - np.min(q_values) + 1e-6)
#             max_q = np.max(q_values)
#             other_q_values = q_values[q_values != max_q]
#             mean_other_q = np.mean(other_q_values) if len(other_q_values) > 0 else 0
#             confidence = float(max_q - mean_other_q)  # Will now be between 0 and 1
#             print(f"Confidence {confidence}")
            
            
#             # Update prediction history
#             prediction_history.append((action, confidence))
#             if len(prediction_history) > SMOOTHING_WINDOW:
#                 prediction_history.pop(0)
            
#             # State change detection
#             # State change detection
#             current_time = time.time()
#             if len(prediction_history) >= SMOOTHING_WINDOW:
#                 recent_predictions = [p[0] for p in prediction_history[-SMOOTHING_WINDOW:]]
#                 recent_confidences = [p[1] for p in prediction_history[-SMOOTHING_WINDOW:]]
                
#                 most_common = max(set(recent_predictions), key=recent_predictions.count)
#                 avg_confidence = np.mean(recent_confidences)
                
#                 if (most_common != last_state and 
#                     recent_predictions.count(most_common) >= SMOOTHING_WINDOW * 0.6 and  # At least 60% agreement
#                     MIN_CONFIDENCE <= avg_confidence):
                    
#                     last_state = most_common
#                     command = map_action_to_command(int(most_common))
#                     last_command = command
#                     last_command_time = current_time
#                     print(f"[{current_time - start_time:.2f}s] State Change: {most_common}, "
#                         f"Confidence: {avg_confidence:.2f}, Command: {command}")
#                     arduino.send_command(command)
                    
#         time.sleep(0.02)  # 20ms sleep

# except KeyboardInterrupt:
#     print("\nStopped by user.")
#     arduino.send_command('S')
    
# finally:
#     arduino.close()
#     board.stop_stream()
#     board.release_session()
#     arduino.send_command('S')
    
#     elapsed_time = time.time() - start_time
#     iterations = len(prediction_history)
#     print(f"\nRun complete:")
#     print(f"- Total time: {elapsed_time:.1f} seconds")
#     print(f"- Iterations: {iterations}")
#     print(f"- Average rate: {iterations/elapsed_time:.1f} Hz")

NEW CODE TESTING WITH SIMULATED SIGNALS

In [39]:
import numpy as np
import time
import socket
from stable_baselines3 import DQN
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from mne.decoding import CSP
import joblib
import tensorflow as tf
import mne
import torch

MIN_CONFIDENCE = 0.3
MAX_CONFIDENCE = 0.8
COMMAND_DURATION = 0.5

# Load the pre-trained DQN model
model = DQN.load("dqn_plasticity_final")
print("DQN model loaded.")

# Load the pre-trained CSP filters
csp_filters = joblib.load('csp_filters_ovr.pkl')
print("CSP filters loaded for real-time use.")

# Load the scaler used during training
scaler = joblib.load('scaler.pkl')
print("Scaler loaded for real-time use.")

class MultiChannelCircularBuffer:
    def __init__(self, num_channels, buffer_size, window_size=1250, overlap_ratio=0.8):
        self.buffer = np.zeros((num_channels, buffer_size))
        self.buffer_size = buffer_size
        self.window_size = window_size
        self.overlap_samples = int(window_size * overlap_ratio)
        self.step_size = window_size - self.overlap_samples
        self.index = 0
        self.last_window_end = 0
        
        print(f"Initialized circular buffer:")
        print(f"- Buffer size: {buffer_size} samples ({buffer_size/250:.1f}s)")
        print(f"- Window size: {window_size} samples ({window_size/250:.1f}s)")
        print(f"- Overlap: {overlap_ratio*100:.0f}% ({self.overlap_samples} samples)")
        print(f"- Step size: {self.step_size} samples ({self.step_size/250:.3f}s)")

    def add_data(self, data):
        """Add new data and return True if enough new samples for next window"""
        num_samples = data.shape[1]
        
        # Add data to buffer
        if self.index + num_samples <= self.buffer_size:
            self.buffer[:, self.index:self.index + num_samples] = data
        else:
            # Handle wrap-around
            end_index = (self.index + num_samples) % self.buffer_size
            self.buffer[:, self.index:] = data[:, :self.buffer_size - self.index]
            self.buffer[:, :end_index] = data[:, self.buffer_size - self.index:]
            
        self.index = (self.index + num_samples) % self.buffer_size
        
        # Check if we have enough new samples since last window
        samples_since_last = (self.index - self.last_window_end) % self.buffer_size
        return samples_since_last >= self.step_size

    def get_window(self):
        """Get the latest overlapping window of data"""
        # Update last window position
        self.last_window_end = self.index
        
        # Calculate start position for window
        start_idx = (self.index - self.window_size) % self.buffer_size
        
        if start_idx < self.index:
            return self.buffer[:, start_idx:self.index]
        else:
            return np.concatenate((
                self.buffer[:, start_idx:],
                self.buffer[:, :self.index]
            ), axis=1)

import keyboard

class FeedbackHandler:
    def __init__(self, model):
        self.model = model
        self.current_features = None
        
        # Keyboard mapping
        self.key_to_action = {
            'left': 0,    # L
            'right': 1,   # R
            'up': 2,      # F
            'down': 3     # B
        }
    
    def check_feedback(self):
        """Check for keyboard feedback"""
        for key, action in self.key_to_action.items():
            if keyboard.is_pressed(key):
                if self.current_features is not None:
                    self.update_model(action)
                    time.sleep(0.2)  # Debounce
                    return True
        return False
    
    def update_model(self, correct_action):
        """Update model with correct action"""
        if self.current_features is not None:
            # Convert action to numpy array
            action_array = np.array([correct_action])
            
            # Add experience to replay buffer
            self.model.replay_buffer.add(
                obs=self.current_features,
                next_obs=self.current_features,  # Same state since this is immediate feedback
                action=action_array,
                reward=1.0,  # Positive reward for correct action
                done=False,
                infos={}
            )
            
            # Train for one gradient step
            if self.model.replay_buffer.size() > self.model.batch_size:
                self.model.train(gradient_steps=1, batch_size=self.model.batch_size)
            
            print(f"Model updated - Correct action was: {map_action_to_command(correct_action)}")
    
    def set_current_features(self, features):
        """Store current features for potential update"""
        self.current_features = features.copy()  # Make a copy to ensure we don't modify the original
# In your main loop:
feedback_handler = FeedbackHandler(model)



class SimulationConnection:
    def __init__(self, host='http://localhost:5000'):
        self.host = host
    
    def send_command(self, command):
        try:
            import requests
            response = requests.get(f"{self.host}/command/{command}")
            return response.json()['status'] == 'ok'
        except Exception as e:
            return False
    
    def close(self):
        pass

simulation = SimulationConnection()

class ArduinoConnection:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.current_command = 'S'
        self.STOP_DURATION = 0.5
        
    def send_command(self, new_command):
        if new_command != self.current_command:
            try:
                # Send stop command
                print("Stopping for 1 second...")
                self._send_http_command('S')
                time.sleep(self.STOP_DURATION)
                
                # Send new command
                self._send_http_command(new_command)
                self.current_command = new_command
                print(f"Changed to new command: {new_command}")
                
            except Exception as e:
                print(f"Connection error: {e}")
    
    def _send_http_command(self, command):
        import requests
        try:
            url = f"http://{self.host}:{self.port}/{command}"
            requests.get(url, timeout=1)
        except Exception as e:
            print(f"HTTP request error: {e}")
    
    def close(self):
        self._send_http_command('S')
arduino = ArduinoConnection('192.168.1.168', 80)

def map_action_to_command(action):
    action_mapping = {
        0: 'L',
        1: 'R',
        2: 'F',
        3: 'B'
    }
    return action_mapping.get(action, 'S')

# Initialize components
num_channels = 6
buffer_size = 1250 * 10  # 10 seconds buffer
window_size = 1250       # 5 seconds window
samples_per_read = 1250  # Read 5 seconds of data at a time

# Initialize the buffer
eeg_buffer = MultiChannelCircularBuffer(
    num_channels=num_channels,
    buffer_size=buffer_size,
    window_size=window_size,
    overlap_ratio=0.65
)
predictions = []
# Set up BrainFlow
params = BrainFlowInputParams()
params.serial_port = 'COM9'  # Replace with your actual COM port
board = BoardShim(BoardIds.CYTON_BOARD.value, params)
board.prepare_session()
print("BrainFlow session prepared.")
board.start_stream()
print("Started data stream from board.")

def get_conservative_prediction(predictions, confidences, similarity_threshold=0.5):
    """
    When confidences are close (within threshold), pick the lower confidence prediction
    """
    if not confidences:  # Empty list check
        return None, None
    
    # Get the highest confidence and its index
    max_conf = max(confidences)
    max_idx = confidences.index(max_conf)
    
    # Look for confidences that are close to the maximum
    close_indices = [i for i, conf in enumerate(confidences) 
                    if (max_conf - conf) <= similarity_threshold]
    
    if len(close_indices) > 1:
        # If we have close confidences, pick the lower one
        min_conf_idx = min(close_indices, key=lambda i: confidences[i])
        return predictions[min_conf_idx], confidences[min_conf_idx]
    
    # If no close confidences, return the highest one
    return predictions[max_idx], confidences[max_idx]


try:
    # Initialize state variables
    prediction_history = []
    last_state = None
    last_command = 'S'
    last_command_time = time.time()
    start_time = time.time()
    model.exploration_rate = 2
    model.exploration_final_eps = 2
    while True:
        # Get new data (5 second chunks)
        data = board.get_current_board_data(samples_per_read)
        eeg_channels = BoardShim.get_eeg_channels(BoardIds.CYTON_BOARD.value)
        eeg_data = data[eeg_channels[:6], :]
        

        # Add data and check if we have enough for a window
        if eeg_buffer.add_data(eeg_data):
            # Get window and process
            window_data = eeg_buffer.get_window().reshape(1, num_channels, -1)
            print(f"Processing window of shape: {window_data.shape}")
            
            # Filter data
            # data_downsampled = mne.filter.resample(window_data, up=2)  # Actually convert to 125Hz
            # filtered_data = mne.filter.filter_data(data_downsampled, sfreq=250, l_freq=0.5, h_freq=45, verbose=False)
            # Initialize CSP features list
            csp_features_list = []
            
            # Apply CSP transform for each class
            for class_id, csp in csp_filters.items():
                transformed = csp.transform(window_data)
                # if hasattr(raw, 'info'):  # Only plot if raw.info is available
                #     csp.plot_patterns(info=raw.info, components=list(range(4)), ch_type='eeg')
                csp_features_list.append(transformed)
            
            # Concatenate CSP features
            csp_ftrs = np.concatenate(csp_features_list, axis=1)
            print(f"CSP Features Shape: {csp_ftrs.shape}")
            
            # Extract features
            ftrs = featuresarray_load(csp_ftrs)
            print(f"Extracted Features Shape: {ftrs.shape}")
            
            predictions = []
            confidences = []

            for i in range(0, 20, 4):
                ftrs_scaled = scaler.transform(ftrs.reshape(-1, ftrs.shape[-1])).reshape(ftrs.shape)
                ftrs_scaled = ftrs_scaled[:, i:i+4, :]
                feedback_handler.set_current_features(ftrs_scaled)
                
                # Get prediction
                action, _ = model.predict(ftrs_scaled, deterministic=False)  # Changed this line
                action = action[0]
                # command = map_action_to_command(action)
                # print(f"Mapped Command: {command} ")
                # simulation.send_command(command)
                # Calculate confidence
                q_values = model.q_net(model.q_net.obs_to_tensor(ftrs_scaled)[0])[0].detach().numpy()
                q_values = (q_values - np.min(q_values)) / (np.max(q_values) - np.min(q_values) + 1e-6)
                
                max_q = np.max(q_values)
                other_q_values = q_values[q_values != max_q]
                confidence = float(max_q - np.mean(other_q_values)) if len(other_q_values) > 0 else 0
                
                # Only add high confidence predictions
                if confidence > 0.6:  # You can adjust this threshold
                    predictions.append(int(action))
                    confidences.append(confidence)

            # Get conservative prediction
            action, confidence = get_conservative_prediction(predictions, confidences)
            if action is not None:
                command = map_action_to_command(action)
                print(f"Mapped Command: {command} {predictions}{confidences}")
            else:
                command = 'S'  # Default to stop if no confident predictions
            simulation.send_command(command)        
            arduino.send_command(command)
            
        time.sleep(0.02)  # 20ms sleep

except KeyboardInterrupt:
    print("\nStopped by user.")    
finally:
    simulation.close()
    board.stop_stream()
    board.release_session()
    simulation.send_command('S')
    arduino.send_command('S')


DQN model loaded.
CSP filters loaded for real-time use.
Scaler loaded for real-time use.
Initialized circular buffer:
- Buffer size: 12500 samples (50.0s)
- Window size: 1250 samples (5.0s)
- Overlap: 65% (812 samples)
- Step size: 438 samples (1.752s)
BrainFlow session prepared.
Started data stream from board.
Processing window of shape: (1, 6, 1250)
CSP Features Shape: (1, 20, 1250)
Extracted Features Shape: (1, 20, 522)
Mapped Command: B [1, 3, 3, 3, 2][0.9008080363273621, 0.6786321401596069, 0.890689492225647, 0.6679656505584717, 0.8210822343826294]
Stopping for 1 second...
Changed to new command: B
Processing window of shape: (1, 6, 1250)
CSP Features Shape: (1, 20, 1250)
Extracted Features Shape: (1, 20, 522)
Mapped Command: R [2, 4, 1, 3][0.9080088138580322, 0.8935555219650269, 0.6532490253448486, 0.9372038245201111]
Stopping for 1 second...
Changed to new command: R
Processing window of shape: (1, 6, 1250)
CSP Features Shape: (1, 20, 1250)
Extracted Features Shape: (1, 20, 522)

BrainFlowError: BOARD_WRITE_ERROR:4 unable to stop streaming session

In [32]:
board.release_all_sessions()

In [38]:
import time

def test_car_movement(arduino):
    try:
        print("Starting car movement test...")
        
        # Forward for 3 seconds
        print("Moving Forward")
        arduino.send_command('F')
        time.sleep(3)
        
        # Turn Left for 2 seconds
        print("Turning Left")
        arduino.send_command('L')
        time.sleep(3)
        
        
        
        # Turn Right for 2 seconds
        print("Turning Right")
        arduino.send_command('R')
        time.sleep(3)
        
        # Backward for 3 seconds
        print("Moving Backward")
        arduino.send_command('B')
        time.sleep(3)
    
        # Stop
        print("Stopping")
        arduino.send_command('S')
        
    except KeyboardInterrupt:
        print("\nTest stopped by user")
        arduino.send_command('S')
    except Exception as e:
        print(f"Error during test: {e}")
        arduino.send_command('S')

# Run the test
arduino = ArduinoConnection('192.168.1.168', 80)
test_car_movement(arduino)
arduino.close()

Starting car movement test...
Moving Forward
Stopping for 1 second...
Changed to new command: F
Turning Left
Stopping for 1 second...
Changed to new command: L
Turning Right
Stopping for 1 second...
Changed to new command: R
Moving Backward
Stopping for 1 second...
Changed to new command: B
Stopping
Stopping for 1 second...
Changed to new command: S


In [ ]:
raw = mne.io.read_raw_bdf("collected_data/session30.bdf")
raw

In [ ]:
def save_as_annotated_fif():
    import mne

    task_duration = 5
    break_duration = 3
    session_start_time = 30

    # Updated task labels dictionary to include all types
    task_labels = {
        "Relax": "relax",
        "Right": "right_hand",
        "Left": "left_hand",
        "Blinking": "blinking",
        "Jaw": "jaw_clenching"
    }

    # Define the path to your BDF files and the annotations file
    bdf_files = [
        "session30.bdf"
    ]
    annotations_file = "collected_data/annotations.txt"

    with open(annotations_file, "r") as f:
        data = f.read().split("_____________________________________________")
    
    # Find section30
    session30_index = next(i for i, section in enumerate(data) if "session30" in section)
    data = data[session30_index:]
    
    print("Total sessions available:", len(data))
    print("Data content:", data)

    # Updated sessions dictionary
    sessions_per_file = {
        "session30.bdf": 1  # Contains session30 with all task types
    }

    for i, bdf_file in enumerate(bdf_files):
        raw = mne.io.read_raw_bdf("collected_data/"+bdf_file, preload=True)
        raw.drop_channels(['EEG 7', 'EEG 8', 'Accel X', 'Accel Y', 'Accel Z'])
        
        # Get tasks for this file
        all_tasks = []
        for j in range(sessions_per_file[bdf_file]):
            if i + j < len(data):
                session_tasks = data[i + j].strip().split("\n")[1:]
                print(f"Adding {len(session_tasks)} tasks from session30")
                all_tasks.extend(session_tasks)

        print(f"Total tasks for {bdf_file}: {len(all_tasks)}")

        onsets = []
        durations = []
        descriptions = []

        current_time = session_start_time
        for task_name in all_tasks:
            # Map the task name to its label
            if task_name in task_labels:
                descriptions.append(task_labels[task_name])
                onsets.append(current_time)
                durations.append(task_duration)
                current_time += task_duration + break_duration
            else:
                print(f"Warning: Unknown task type '{task_name}'")

        annotations = mne.Annotations(onset=onsets, duration=durations, description=descriptions)
        print(f"Created {len(annotations)} annotations for {bdf_file}")
        raw.set_annotations(annotations)

        annotated_file = bdf_file.replace(".bdf", "_annotated.fif")
        raw.save("collected_data/"+annotated_file, overwrite=True)
        print(f"Saved {annotated_file} with {len(raw.annotations)} annotations\n")

def combine_fifs():
    import mne
    import os

    data_folder = 'collected_data'
    # Modified to only include session30
    bdf_files = [os.path.join(data_folder, 'session30_annotated.fif')]
    print("Files to combine:", bdf_files)
    
    raw_combined = None

    for bdf_file in bdf_files:
        raw = mne.io.read_raw_fif(bdf_file, preload=True)
        print(f"Loaded {bdf_file} with {len(raw.annotations)} annotations")
        
        if raw_combined is None:
            raw_combined = raw
        else:
            raw_combined.append(raw)
            print(f"After append: {len(raw_combined.annotations)} total annotations")

    # Save to a new file name to avoid overwriting existing S02.fif
    combined_bdf_path = os.path.join("formatted_data", 'S03.fif')
    raw_combined.save(combined_bdf_path, overwrite=True)
    print(f"\nSaved {combined_bdf_path} with {len(raw_combined.annotations)} total annotations")

# Run both functions
save_as_annotated_fif()
combine_fifs()

In [ ]:
import mne
import numpy as np
from mne.decoding import CSP
import matplotlib.pyplot as plt

# Load the combined .fif file
raw = mne.io.read_raw_fif("formatted_data/S02.fif", preload=True)

# Rename channels
mapping = {
    'EEG 1': 'FCz',
    'EEG 2': 'C3',
    'EEG 3': 'Cz',
    'EEG 4': 'CPz',
    'EEG 5': 'C2',
    'EEG 6': 'C4'
}
raw.rename_channels(mapping)

# Set montage
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

# Extract events and event IDs
events, event_id = mne.events_from_annotations(raw)
print('event_id', event_id)

# Create epochs
tmin, tmax = 0, 5
epochs = mne.Epochs(
    raw, events, event_id=event_id,
    tmin=tmin, tmax=tmax, baseline=None, preload=True
)

# Get data and labels
X = epochs.get_data()
y = np.zeros(len(epochs.events))
for i, event in enumerate(epochs.events[:, -1]):
    if event == event_id['blinking']:        # 1 -> 2
        y[i] = 2
    elif event == event_id['jaw_clenching']: # 2 -> 3
        y[i] = 3
    elif event == event_id['left_hand']:     # 3 -> 0
        y[i] = 0
    elif event == event_id['relax']:         # 4 -> 4
        y[i] = 4
    elif event == event_id['right_hand']:    # 5 -> 1
        y[i] = 1

# Create CSP for each class (One-vs-Rest)
class_names = ['Left Hand', 'Right Hand', 'Blinking', 'Jaw Clenching', 'Relax']
n_components = 6  # Use all components since we have 6 channels

plt.figure(figsize=(20, 15))

for class_idx in range(5):
    # Create binary labels for current class
    binary_y = (y == class_idx).astype(int)
    
    # Fit CSP
    csp = CSP(n_components=n_components, reg=None, log=True, norm_trace=False)
    csp.fit(X, binary_y)
    
    # Plot CSP patterns
    plt.subplot(2, 3, class_idx + 1)
    mne.viz.plot_topomap(csp.patterns_[:, 0], epochs.info, axes=plt.gca(), 
                        show=False, contours=0)
    plt.title(f'CSP Pattern for {class_names[class_idx]}')

# Adjust layout and display
plt.tight_layout()
plt.show()

# Plot CSP patterns with multiple components
n_components_to_plot = 3  # Plot top 3 components for each class

fig, axes = plt.subplots(5, n_components_to_plot, figsize=(15, 25))
fig.suptitle('CSP Patterns: Top 3 Components for Each Class', fontsize=16, y=1.02)

for class_idx in range(5):
    # Create binary labels for current class
    binary_y = (y == class_idx).astype(int)
    
    # Fit CSP
    csp = CSP(n_components=n_components, reg=None, log=True, norm_trace=False)
    csp.fit(X, binary_y)
    
    # Plot top components
    for comp_idx in range(n_components_to_plot):
        ax = axes[class_idx, comp_idx]
        mne.viz.plot_topomap(csp.patterns_[:, comp_idx], epochs.info, axes=ax, 
                            show=False, contours=0)
        if comp_idx == 0:
            ax.set_title(f'{class_names[class_idx]}\nComponent {comp_idx+1}')
        else:
            ax.set_title(f'Component {comp_idx+1}')

plt.tight_layout()
plt.show()

# Print class counts
print("\nNumber of trials per class:")
for i, name in enumerate(class_names):
    print(f"{name}: {np.sum(y == i)}")

In [ ]:
def get_model_size(model):
    # Get total number of parameters
    param_size = 0
    buffer_size = 0
    
    # For each parameter, calculate size in bytes
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    
    # Buffer size (for gradients, etc.)
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    # Convert to MB
    size_mb = (param_size + buffer_size) / 1024**2
    
    return {
        'parameters_size_mb': param_size / 1024**2,
        'buffer_size_mb': buffer_size / 1024**2,
        'total_size_mb': size_mb
    }

# For your model:
model_size = get_model_size(model.q_net)
print(f"Model Memory Usage:")
print(f"Parameters: {model_size['parameters_size_mb']:.2f} MB")
print(f"Buffers: {model_size['buffer_size_mb']:.2f} MB")
print(f"Total: {model_size['total_size_mb']:.2}")